# Multi-Vendor Laptop Spec Batch Parser (Lenovo + HP)

This notebook processes multiple PDF spec sheets (current support: Lenovo ThinkPad E-series style & HP ProBook 4xx G11 style) and normalizes them into a single DataFrame.

## Workflow
1. Define list of PDF paths (`PDF_PATHS`).
2. Convert PDFs to markdown via Docling.
3. Auto-detect vendor (Lenovo vs HP) and parse.
4. Normalize to canonical schema:
   - brand
   - model
   - Processor
   - Operating System
   - Graphics
   - Chipset
   - Memory (RAM)
   - Storage
   - Display
   - External Monitor Support
   - Audio
   - Camera
   - Input Devices
   - Dimensions & Weight
   - Case / Chassis
   - Ports
   - Card Reader
   - Wireless Networking
   - Wired Networking
   - Mobile Broadband
   - Docking
   - Battery
   - Power Adapter
   - Biometric Security
   - General Security
   - Software & Management
   - Warranty
   - Environmental & Durability Standards

5. Export CSV / JSON.

You can extend mapping logic for additional vendors or new field naming variations.

---

## 0. Install Dependencies (Run Once If Needed)
Uncomment the pip line if packages are not present. Docling may require system dependencies depending on environment.

In [1]:
# %pip install pandas docling tqdm

## 1. Imports & Global Schema / Utilities

In [ ]:
import re
from typing import List, Dict, Tuple
import pandas as pd
from pathlib import Path
from dataclasses import dataclass
from tqdm import tqdm
from docling.document_converter import DocumentConverter

CANON_COLUMNS = [
    "Brand","Model","Processor","Operating System","Graphics","Chipset","Memory (RAM)","Storage","Display",
    "External Monitor Support","Audio","Camera","Input Devices","Dimensions & Weight","Case / Chassis","Ports",
    "Card Reader","Wireless Networking","Wired Networking","Mobile Broadband","Docking","Battery","Power Adapter",
    "Biometric Security","General Security","Software & Management","Warranty","Environmental & Durability Standards"
]

def normalize_space(text: str) -> str:
    return re.sub(r'\s+', ' ', text.strip()) if text else text

def normalize_key(key: str) -> str:
    return re.sub(r'[^a-z0-9]+', ' ', key.lower()).strip()

def dedup_join(parts: List[str], sep='\n') -> str:
    seen = set()
    out = []
    for p in parts:
        p_strip = p.strip()
        if not p_strip: continue
        if p_strip in seen: continue
        seen.add(p_strip)
        out.append(p_strip)
    return sep.join(out).strip()

def safe_append(target: Dict[str,str], key: str, value: str, sep='\n'):
    if not value: return
    if key not in target or not target[key].strip():
        target[key] = value.strip()
    else:
        if value.strip() not in target[key]:
            target[key] = target[key].rstrip() + sep + value.strip()

def extract_table_blocks(markdown_text: str) -> List[str]:
    lines = markdown_text.splitlines()
    tables = []
    current = []
    in_table = False
    for ln in lines:
        if '|' in ln:
            current.append(ln)
            in_table = True
        else:
            if in_table:
                if len(current) >= 2:
                    tables.append('\n'.join(current))
                current = []
                in_table = False
    if in_table and len(current) >= 2:
        tables.append('\n'.join(current))
    return tables

def parse_simple_key_value_table(table_md: str):
    """
    Parse generic 2+ column markdown tables into (key, value) pairs.
    Fix: Do NOT automatically skip the first line; only skip pure separator lines.
    This allows single-row tables (like the HP 'Available Operating Systems') to be captured.
    """
    rows = []
    lines = [l for l in table_md.split('\n') if l.strip()]
    
    if not lines:
        return rows

    # Regex for a separator line made only of dashes and pipes (e.g. |-----|----|)
    sep_re = re.compile(r'^\|(?:\s*-+\s*\|)+\s*$')

    for ln in lines:
        if sep_re.match(ln):
            continue  # skip separator
        parts = [c.strip() for c in ln.strip().strip('|').split('|')]
        if len(parts) < 2:
            continue
        key = parts[0]
        value = ' | '.join(parts[1:]).strip()
        if key and value:
            rows.append((key, value))

    # Fallback: if nothing parsed but there are at least 2 cells in first line, treat that as a single row
    if not rows and lines:
        parts = [c.strip() for c in lines[0].strip().strip('|').split('|')]
        if len(parts) >= 2:
            rows.append((parts[0], ' | '.join(parts[1:]).strip()))
    return rows

@dataclass
class ParseDiagnostics:
    vendor: str
    file: str
    unmatched_keys: List[str]
    notes: List[str]

ALL_DIAGNOSTICS: List[ParseDiagnostics] = []

## 2. Lenovo Parsing Logic (Heading-Based)
Patterns are broad to handle minor variation among Lenovo PDFs.

In [16]:
LENOVO_PATTERNS = {
    "Processor": ["processor"],
    "Operating System": ["operating system"],
    "Graphics": ["graphics"],
    "Chipset": ["chipset"],
    "Memory (RAM)": ["max memory","memory slots","memory type","memory"],
    "Storage": ["max storage support","storage slot","storage type","storage"],
    "Display": ["display"],
    "External Monitor Support": ["monitor support"],
    "Audio": ["audio chip","speakers","microphone"],
    "Camera": ["camera"],
    "Input Devices": ["keyboard","keyboard backlight","ultranav","pen"],
    "Dimensions & Weight": ["dimensions","weight"],
    "Ports": ["standard ports","ports"],
    "Wireless Networking": ["wlan + bluetooth","wlan","bluetooth"],
    "Wired Networking": ["ethernet"],
    "Docking": ["docking"],
    "Battery": ["battery"],
    "Power Adapter": ["power adapter"],
    "Biometric Security": ["fingerprint"],
    "General Security": ["security chip","physical locks","bios security","other security"],
    "Software & Management": ["system management"],
    "Warranty": ["base warranty"],
    "Environmental & Durability Standards": ["green certifications","mil spec test","sustainability"]
}

def extract_headings_lenovo(md: str):
    pattern = re.compile(r'^(#{1,6})\s+(.+?)\s*$', re.MULTILINE)
    headings = []
    for m in pattern.finditer(md):
        level = len(m.group(1))
        title = m.group(2).strip()
        start = m.start()
        line_end = md.find('\n', m.end())
        content_start = len(md) if line_end == -1 else line_end + 1
        headings.append({
            'level': level,
            'raw': title,
            'norm': normalize_key(title),
            'start': start,
            'content_start': content_start
        })
    for i, h in enumerate(headings):
        h['content_end'] = headings[i+1]['start'] if i < len(headings)-1 else len(md)
        h['content'] = md[h['content_start']:h['content_end']].strip()
    return headings

def parse_lenovo(md: str, file: str) -> Dict[str,str]:
    headings = extract_headings_lenovo(md)
    data = {c: '' for c in CANON_COLUMNS}
    data['Brand'] = 'Lenovo'
    model_candidates = re.findall(r'^(?:#+\s*)?(ThinkPad[^\n]{0,80})$', md, flags=re.MULTILINE | re.IGNORECASE)
    model = ''
    for c in model_candidates:
        if re.search(r'gen\s*\d', c, re.I):
            model = normalize_space(c)
            break
    data['Model'] = re.sub(r'(?i)thinkpad', 'ThinkPad', model)

    for field, pats in LENOVO_PATTERNS.items():
        blocks = []
        for h in headings:
            if any(p in h['norm'] for p in pats):
                txt = f"{h['raw']}\n{h['content']}".strip()
                if txt: blocks.append(txt)
        if blocks:
            data[field] = dedup_join(blocks, '\n\n')

    # Case / Chassis
    case_color = re.search(r'Case Color[\s\S]*?(?:\n##|$)', md)
    case_material = re.search(r'Case Material[\s\S]*?(?:\n##|$)', md)
    parts = []
    if case_color: parts.append(case_color.group(0).strip())
    if case_material: parts.append(case_material.group(0).strip())
    data['Case / Chassis'] = dedup_join(parts)

    # Card Reader
    m_card = re.search(r'Card Reader\n([^\n]+)', md)
    if m_card: data['Card Reader'] = m_card.group(1).strip()

    # Mobile Broadband (WWAN)
    m_wwan = re.search(r'WWAN\n([^\n]+)', md)
    if m_wwan: data['Mobile Broadband'] = m_wwan.group(1).strip()

    # Biometric Security (IR camera mention)
    if 'ir camera' in md.lower():
        safe_append(data, 'Biometric Security', 'Optional IR camera (Windows Hello)')

    # External Monitor fallback
    if not data['External Monitor Support'] and data['Ports']:
        if re.search(r'hdmi|displayport|usb-c', data['Ports'], re.I):
            data['External Monitor Support'] = 'External display via HDMI / USB-C (DisplayPort)'

    return data

## 3. HP Parsing Logic (Table + Narrative)
Captures large tables and splits communications into wireless/wired/mobile categories. Unmatched table keys are logged for diagnostics.

In [17]:
HP_KEY_MAP = {
    'available operating systems': ('Operating System', None),
    'processor family': ('Processor', 'family'),
    'available processors': ('Processor', 'models'),
    'maximummemory': ('Memory (RAM)', 'max'),
    'memoryslots': ('Memory (RAM)', 'slots'),
    'memory slots': ('Memory (RAM)', 'slots'),
    'internal storage': ('Storage', 'internal'),
    'display size diagonal metric': ('Display', 'size'),
    'display': ('Display', 'panels'),
    'available graphics': ('Graphics', None),
    'audio': ('Audio', None),
    'ports and connectors': ('Ports', None),
    'input devices': ('Input Devices', None),
    'communications': ('_communications', None),
    'camera': ('Camera', None),
    'software': ('Software & Management', 'software'),
    'securitymanagement': ('General Security', 'mgmt'),
    'security management': ('General Security', 'mgmt'),
    'security software licenses': ('Software & Management', 'licenses'),
    'fingerprint reader': ('Biometric Security', 'fingerprint'),
    'management features': ('Software & Management', 'mgmtfeatures'),
    'memorycarddevice': ('Card Reader', None),
    'memory card device': ('Card Reader', None),
    'power': ('Power Adapter', None),
    'battery type': ('Battery', None),
    'dimensions': ('Dimensions & Weight', 'dims'),
    'weight': ('Dimensions & Weight', 'weight'),
    'ecolabels': ('Environmental & Durability Standards', 'ecolabels'),
    'energy star certified': ('Environmental & Durability Standards', 'energystar'),
    'certification and compliance': ('Environmental & Durability Standards', 'compliance'),
    'sustainable impact specifications': ('Environmental & Durability Standards', 'sustainability'),
    'warranty': ('Warranty', None)
}

def parse_hp(md: str, file: str) -> Dict[str,str]:
    data = {c: '' for c in CANON_COLUMNS}
    data['Brand'] = 'HP'
    model_match = re.search(r'^(?:#+\s*)?(HP\s+ProBook[^\n]{0,80})$', md, flags=re.MULTILINE)
    if model_match:
        data['Model'] = normalize_space(model_match.group(1))

    tables = extract_table_blocks(md)
    kv_pairs = []
    for tbl in tables:
        kv_pairs.extend(parse_simple_key_value_table(tbl))

    tmp_store = {
        'Processor': {},
        'Memory (RAM)': {},
        'Display': {},
        'Dimensions & Weight': {},
        'Environmental & Durability Standards': {},
        'Software & Management': {}
    }
    communications_raw = []
    unmatched = []


    for raw_key, raw_val in kv_pairs:
        original_key = raw_key
        # Base normalization
        nkey = normalize_key(raw_key)

        # 1. Remove trailing footnote digit tokens (e.g., "processor family 5" -> "processor family")
        nkey_footnote_stripped = re.sub(r'(?:\b\d+\b[ ,]*)+$', '', nkey).strip()

        # 2. Also create a version with ALL standalone digit tokens removed (handles embedded patterns)
        nkey_no_digits = re.sub(r'\b\d+\b', '', nkey_footnote_stripped).strip()

        # 3. Choose the first that matches our map
        candidate_keys = [nkey, nkey_footnote_stripped, nkey_no_digits]

        resolved_key = None
        for ck in candidate_keys:
            if ck in HP_KEY_MAP:
                resolved_key = ck
                break

        # 4. If still not found, try prefix (startswith) fuzzy match
        if not resolved_key:
            for map_key in HP_KEY_MAP.keys():
                if ck.startswith(map_key):
                    resolved_key = map_key
                    break

        val_clean = raw_val.strip()
        # spacing fixes for jammed alphanumerics
        val_clean = re.sub(r'(?<=\d)(?=[A-Za-z])', ' ', val_clean)
        val_clean = re.sub(r'(?<=[a-zA-Z])(?=\d)', ' ', val_clean)

        if resolved_key:
            canon, sub = HP_KEY_MAP[resolved_key]
            if canon == '_communications':
                communications_raw.append(val_clean)
                continue
            if sub:
                tmp_store.setdefault(canon, {})[sub] = val_clean
            else:
                if canon in tmp_store:
                    tmp_store[canon]['main'] = val_clean
                else:
                    safe_append(data, canon, val_clean)
        else:
            unmatched.append(original_key)

    # Processor synthesis
    if tmp_store['Processor']:
        parts = []
        fam = tmp_store['Processor'].get('family')
        if fam: parts.append('Family: ' + fam)
        models = tmp_store['Processor'].get('models')
        if models: parts.append('Models: ' + models)
        data['Processor'] = dedup_join(parts)

    # Memory
    if tmp_store['Memory (RAM)']:
        mparts = []
        if 'max' in tmp_store['Memory (RAM)']:
            mparts.append('Maximum: ' + tmp_store['Memory (RAM)']['max'])
        if 'slots' in tmp_store['Memory (RAM)']:
            mparts.append('Slots: ' + tmp_store['Memory (RAM)']['slots'])
        data['Memory (RAM)'] = dedup_join(mparts)

    # Storage formatting
    if data['Storage']:
        data['Storage'] = data['Storage'].replace('internal storage', 'Internal Storage:')

    # Display
    if tmp_store['Display']:
        dparts = []
        if 'size' in tmp_store['Display']:
            dparts.append('Size: ' + tmp_store['Display']['size'])
        if 'panels' in tmp_store['Display']:
            dparts.append('Options: ' + tmp_store['Display']['panels'])
        data['Display'] = dedup_join(dparts)

    # Dimensions & Weight
    if tmp_store['Dimensions & Weight']:
        dw = []
        if 'dims' in tmp_store['Dimensions & Weight']:
            dw.append('Dimensions: ' + tmp_store['Dimensions & Weight']['dims'])
        if 'weight' in tmp_store['Dimensions & Weight']:
            dw.append('Weight: ' + tmp_store['Dimensions & Weight']['weight'])
        data['Dimensions & Weight'] = dedup_join(dw)

    # Environmental & Durability Standards
    if tmp_store['Environmental & Durability Standards']:
        env_map = tmp_store['Environmental & Durability Standards']
        env_parts = []
        label_map = {
            'ecolabels': 'Ecolabels',
            'energystar': 'Energy Star',
            'compliance': 'Compliance',
            'sustainability': 'Sustainability'
        }
        for k, label in label_map.items():
            if k in env_map:
                env_parts.append(f"{label}: {env_map[k]}")
        if re.search(r'MIL-STD', md, re.I):
            env_parts.append('MIL-STD Tested (marketing claim)')
        data['Environmental & Durability Standards'] = dedup_join(env_parts)

    # Software & Management
    sm = tmp_store['Software & Management']
    if sm:
        sm_parts = []
        if 'software' in sm: sm_parts.append('Software: ' + sm['software'])
        if 'licenses' in sm: sm_parts.append('Security Licenses: ' + sm['licenses'])
        if 'mgmtfeatures' in sm: sm_parts.append('Management Features: ' + sm['mgmtfeatures'])
        data['Software & Management'] = dedup_join(sm_parts)

    # Communications splitting
    if communications_raw:
        comm_text = ' '.join(communications_raw)
        wireless_matches = re.findall(r'(Wi-?Fi[^;]+Bluetooth[^;]+|Wi-?Fi[^;]+)', comm_text, flags=re.I)
        if wireless_matches:
            safe_append(data, 'Wireless Networking', dedup_join([normalize_space(w) for w in wireless_matches]))
        if re.search(r'rj-?45', comm_text, re.I):
            safe_append(data, 'Wired Networking', 'RJ-45 Ethernet (Gigabit)')
        if re.search(r'LTE|WWAN|Cat-?M1|Cat16|5G', comm_text, re.I):
            lte_part = ' '.join(re.findall(r'(?:Qualcomm|LTE|WWAN|Cat-?M1|4G\s*LTE|Cat16)[^;]*', comm_text, flags=re.I))
            safe_append(data, 'Mobile Broadband', normalize_space(lte_part))

    # External Monitor Support inference from Ports
    if data['Ports']:
        ext_parts = []
        if re.search(r'HDMI\s*2\.1', data['Ports'], re.I): ext_parts.append('HDMI 2.1')
        if re.search(r'DisplayPort', data['Ports'], re.I): ext_parts.append('DisplayPort 1.4 over USB-C')
        if ext_parts:
            data['External Monitor Support'] = dedup_join(ext_parts, '; ')

    # Biometric Security (IR camera mention)
    if re.search(r'5 ?MP\s*IR', md, re.I):
        safe_append(data, 'Biometric Security', 'Optional 5MP IR Camera (Windows Hello)')

    # Case / Chassis from narrative (aluminum references)
    chassis_sentences = re.findall(r'[^\n.]*aluminum[^\n.]*[\.]?', md, flags=re.I)
    if chassis_sentences:
        data['Case / Chassis'] = dedup_join([normalize_space(s) for s in chassis_sentences], ' ')

    # Docking inference
    if re.search(r'universal docking solution', md, re.I):
        data['Docking'] = 'Supports universal USB-C docking (two multifunction USB-C ports)'
    elif 'USB' in data['Ports']:
        safe_append(data, 'Docking', 'USB-C multi-function ports (PD/DP) enable docking')

    # Chipset not specified; leave blank
    if data['Card Reader'] and 'nano sim' in data['Card Reader'].lower():
        data['Card Reader'] = data['Card Reader'] + ' (Nano SIM / WWAN option)'

    # Diagnostics collection
    diag = ParseDiagnostics(
        vendor='HP', file=file,
        unmatched_keys=sorted(set(unmatched)),
        notes=[]
    )
    ALL_DIAGNOSTICS.append(diag)

    return data

## 4. Vendor Dispatcher & Batch Processor
Detect vendor (Lenovo/HP) by keywords. Extend logic to new vendors as needed.

In [18]:
def detect_vendor(markdown_text: str) -> str:
    low = markdown_text.lower()
    if 'thinkpad' in low or 'lenovo' in low:
        return 'lenovo'
    if 'hp probook' in low or re.search(r'\bhp\b', low):
        return 'hp'
    return 'unknown'

def parse_spec(markdown_text: str, file: str) -> Dict[str,str]:
    vendor = detect_vendor(markdown_text)
    if vendor == 'lenovo':
        record = parse_lenovo(markdown_text, file)
        diag = ParseDiagnostics(vendor='Lenovo', file=file, unmatched_keys=[], notes=[])
        ALL_DIAGNOSTICS.append(diag)
    elif vendor == 'hp':
        record = parse_hp(markdown_text, file)
    else:
        raise ValueError(f"Unknown vendor for file {file}")
    for c in CANON_COLUMNS:
        record.setdefault(c, '')
    return record

def specs_to_dataframe(markdown_docs: List[Tuple[str,str]]) -> pd.DataFrame:
    # markdown_docs: list of (file_path, markdown_text)
    rows = []
    for file_path, md in markdown_docs:
        rows.append(parse_spec(md, file_path))
    df = pd.DataFrame(rows, columns=CANON_COLUMNS)
    return df

## 5. PDF → Markdown Conversion (Docling Integration)
Define your four PDF paths in `PDF_PATHS`. The code converts each to markdown. Any conversion failure logs an error and skips that file (unless `STRICT=True`).

In [19]:
# UPDATE these with your actual PDF file names/paths.
PDF_PATHS = [
    'ThinkPad_E14_Gen_5_AMD_Spec.pdf',  # Lenovo PDF 1
    'ThinkPad_E14_Gen_5_Intel_Spec.pdf', # Lenovo PDF 2
    'HP ProBook 440.pdf',       # HP PDF 1
    'HP ProBook 450.pdf'        # HP PDF 2
]

STRICT = False  # If True, raise on first failure.

def convert_pdfs_to_markdown(paths: List[str]) -> List[Tuple[str,str]]:
    conv = DocumentConverter()
    results = []
    for p in tqdm(paths, desc='Converting PDFs'):
        path_obj = Path(p)
        if not path_obj.is_file():
            msg = f"File not found: {p}"
            print(msg)
            if STRICT: raise FileNotFoundError(msg)
            continue
        try:
            doc = conv.convert(str(path_obj)).document
            md = doc.export_to_markdown()
            results.append((str(path_obj), md))
        except Exception as e:
            msg = f"Conversion failed for {p}: {e}"
            print(msg)
            if STRICT: raise
    return results

markdown_docs = convert_pdfs_to_markdown(PDF_PATHS)
print(f"Converted {len(markdown_docs)} / {len(PDF_PATHS)} PDFs successfully.")

Converting PDFs:   0%|          | 0/4 [00:00<?, ?it/s]2025-09-27 13:24:01,277 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]


2025-09-27 13:24:01,292 - INFO - Going to convert document batch...
2025-09-27 13:24:01,293 - INFO - Initializing pipeline for StandardPdfPipeline with options hash e647edf348883bed75367b22fbe60347
2025-09-27 13:24:01,295 - INFO - Accelerator device: 'cpu'
2025-09-27 13:24:04,026 - INFO - Accelerator device: 'cpu'
2025-09-27 13:24:05,662 - INFO - Accelerator device: 'cpu'
2025-09-27 13:24:07,643 - INFO - Processing document ThinkPad_E14_Gen_5_AMD_Spec.pdf
C:\Users\Bhagya Dissanayake\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\Bhagya Dissanayake\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_

Converted 4 / 4 PDFs successfully.


## 6. Parse All Documents into a Unified DataFrame

In [20]:
df = specs_to_dataframe(markdown_docs)
pd.set_option('display.max_colwidth', 120)
df

,Brand,Model,Processor,Operating System,Graphics,Chipset,Memory (RAM),Storage,Display,External Monitor Support,...,Wired Networking,Mobile Broadband,Docking,Battery,Power Adapter,Biometric Security,General Security,Software & Management,Warranty,Environmental & Durability Standards
0,Lenovo,ThinkPad E14 Gen 5 (AMD),Processor\n\nProcessor Family\nAMD Ryzen™ 3 / 5 / 7 Processor\n\nProcessor **\n| Processor Name | Cores | Th...,Operating System\n\nOperating System **\n- Windows® 11 Pro ·\n- Windows® 11 Home ·\n- Windows® 11 Home Single Langua...,Graphics\n\nGraphics\n| Graphics | Type | Memory | TGP | Key Features |\n|----------...,Chipset\n\nChipset\nAMD SoC (System on Chip) platform,Memory\n\nMax Memory [1]\nUp to 40GB (8GB soldered + 32GB SO-DIMM) DDR4-3200\n\nMemory Slots\nOne memory soldered to...,"Storage\n\nMax Storage Support [1]\nUp to two drives, 2x M.2 SSD\n\n<!-- image -->\n\nStorage Slot [2]\nTwo M.2 slot...",Display\nDisplay ** [1]\n\n| Size | Resolution | Touch | Type | Brightness | Surface | Aspec...,Monitor Support\n\nMonitor Support\nSupports up to 4 independent displays (native display and 3 external monitors vi...,...,"Ethernet\nGigabit Ethernet, Realtek® RTL8111H-CG, 1x RJ-45, supports Wake-on-LAN",,"Docking\n\nDocking\nVarious docking solutions supported via USB-C®.\n\nFor more compatible docking solutions, please...","Battery\n\nBattery ** [1]\n- 47Wh Rechargeable Li-ion Battery, supports Rapid Charge (charge up to 80% in 1hr) ·\n- ...","Power Adapter\n\nPower Adapter ** [1]\n- 65W USB-C® (2-pin) AC adapter, supports PD 3.0, 100-240V, 50-60Hz ·\n- 65W ...",Fingerprint Reader\n- Touch style fingerprint reader integrated in power button ·\n- No fingerprint reader ·\nOption...,"Security Chip **\n- Firmware TPM 2.0 integrated in SoC ·\n- Discrete TPM 2.0, TCG certified, FIPS 140-2 certified ·\...",System Management\nSystem Management\n\nNon-DASH,Base Warranty **\n- 1-year mail-in service ·\n- 1-year courier or carry-in service ·\n- 1-year courier or carry-in w...,Sustainability\n\nGreen Certifications [1]\n\nGreen Certifications [2]\n- ENERGY STAR® 8.0 ·\n- EPEAT™ Gold Register...
1,Lenovo,ThinkPad E14 Gen 5 (Intel),"Processor\n\nProcessor Family\n13th Generation Intel® U, P or H Series Core i3 / i5 / i7 Processor\n\nProcessor **\n...",Operating System\n\nOperating System **\n- Windows® 11 Pro ·\n- Windows® 11 Home ·\n- Windows® 11 Home Single Langua...,Graphics\n\nGraphics **\n| Graphics | Type | Memory | TGP | Key Features |\...,Chipset\n\nChipset\nIntel® SoC (System on Chip) platform,Memory\n\nMax Memory\n- Up to 40GB (8GB soldered + 32GB SO-DIMM) DDR4-3200 ·\n- Up to 48GB (16GB soldered + 32GB SO-...,"Storage\n\nMax Storage Support [1]\nUp to two drives, 2x M.2 SSD\n\n- M.2 2242 SSD up to 1TB each\n\nStorage Slot [2...",Display\nDisplay ** [1]\n\n| Size | Resolution | Touch | Type | Brightness | Surface | Aspec...,Monitor Support\n\nMonitor Support\nSupports up to 4 independent displays (native display and 3 external monitors vi...,...,"Ethernet\nGigabit Ethernet, Intel® Ethernet Connection I219-V, 1x RJ-45, supports Wake-on-LAN",,Docking\n<!-- image -->\n\nDocking\nVarious docking solutions supported via Thunderbolt™ or USB-C®. For more compati...,"Battery\n** [1]\n\nBattery\n\n- 47Wh Rechargeable Li-ion Battery, supports Rapid Charge (charge up to 80% in 1hr) ·\...","Power Adapter\n\nPower Adapter ** [1]\n- 65W USB-C® (2-pin) AC adapter, supports PD 3.0, 100-240V, 50-60Hz ·\n- 65W ...",Fingerprint Reader\n- Touch style fingerprint reader integrated in power button ·\n- No fingerprint reader ·\nOption...,"Security Chip **\n- Firmware TPM 2.0 integrated in SoC ·\n- Discrete TPM 2.0, TCG certified, FIPS 140-2 certified ·\...",System Management\n\nSystem Management [1]\n- Intel® vPro® Enterprise ·\n- Non-vPro® ·,Base Warranty **\n- 1-year mail-in service ·\n- 1-year courier or carry-in service ·\n- 1-year courier or carry-in w...,Sustainability\n\nGreen Certifications [1]\n\nGreen Certifications [2]\n- ENERGY

In [ ]:
# Saving to CSV
df.to_csv('laptop_tech_specs.csv', index=False)